In [1]:
from __future__ import division
import pickle
import os
import types
import random
import uuid
import math
from copy import deepcopy as copy
import logging


import gym
from gym import spaces
# from gym.envs.robotics import rendering
import numpy as np
import tensorflow as tf
from scipy.misc import logsumexp
import baselines.common.tf_util as U
import baselines.her.rollout as roll
import baselines.her.experiment.config as config
from baselines import logger
import pickle as pkl




In [2]:
%load_ext autoreload

In [3]:
%autoreload

In [4]:
from matplotlib import pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
%matplotlib inline

In [5]:
import matplotlib as mpl
mpl.rc('savefig', dpi=300)
mpl.rc('text', usetex=False)

In [6]:
# logger = logging.getLogger('root')
# print(logger.hasHandlers())
# assert len(logger.handlers) == 1
# handler = logger.handlers[0]
# handler.setLevel(logging.WARNING)

In [7]:
data_dir = os.path.join('data', '5.0')

In [8]:
sess = tf.Session()

create envs, pilot policies

In [9]:
# n_act_dim = 6
# n_obs_dim = 9

In [10]:
max_ep_len = 1000

In [11]:
env = gym.make('FetchSlide-v1')
env.reset()
obs, _, _, info = env.step(env.action_space.sample())

dims = {'o': obs['observation'].shape[0],
    'u': env.action_space.shape[0],
    'g': obs['desired_goal'].shape[0]}

In [12]:
# train_goals = np.arange(1, 9, 1)
n_train_tasks = 10
goals = [env.env._sample_goal() for _ in range(n_train_tasks)]

In [13]:
def make_env_with_goal(env_name, goal):
    def make_env():
        env = gym.make(env_name)
        env.env.goal = goal
        return env
    return make_env

In [14]:
train_envs = [env]
for goal in goals:
    new_env = gym.make('FetchSlide-v1')
    new_env.env.goal = goal
    train_envs += [new_env]

In [15]:
policy = pkl.load(open(data_dir + "/slide_policy.pkl", "rb"))
# policy = config.configure_existing_ddpg(policy, params=params)
# policy._config_replay_buffer()

Creating a DDPG agent with action space 4 x 1.0...


In [16]:
# def make_lander_env(fps=fps, goal=None):
#   env = gym.make('LunarLanderContinuous-v2')
#   env.unwrapped.goal = goal
#   env.action_space = spaces.Discrete(n_act_dim)
#   env.unwrapped._step_orig = env.unwrapped.step
#   def step(self, action):
#     obs, r, done, info = self._step_orig(disc_to_cont(action))
#     return obs, r, done, info
#   env.unwrapped.step = types.MethodType(step, env.unwrapped)
#   env.unwrapped.fps = fps
#   return env

In [17]:
# train_newton_envs = [make_lander_env(fps=fast_fps, goal=goal) for goal in train_goals]

In [18]:
#  def run_ep(policy, env, max_ep_len=max_ep_len, render=False, task_idx=None):
#   obs = env.reset()
#   done = False
#   totalr = 0.
#   prev_obs = obs
#   rollout = []
#   for step_idx in range(max_ep_len+1):
#     if done:
#       break
#     action = policy(obs)
#     obs, r, done, info = env.step(action)
#     rollout.append((prev_obs, action, r, obs, float(done), task_idx))
#     prev_obs = obs
#     if render:
#       env.render()
#     totalr += r
#   return rollout

T = env._max_episode_steps
make_envs = [make_env_with_goal('FetchSlide-v1', goal) for goal in goals]
workers = [roll.RolloutWorker(make_env, policy, dims, logger, T, goals=[goals[i]]) for i,make_env in enumerate(make_envs)]

In [19]:
env.action_space #.contains(np.int64(1))

Box(4,)

fit internal dynamics model

In [20]:
n_train_rollouts_per_env = 10

In [21]:
# Don't have best expert, ensure all demos are successes

demo_rollouts = []
max_attempt = 100

for i,worker in enumerate(workers):
    worker_demos = []
    print(i)
    for _ in range(n_train_rollouts_per_env):
        rollout, success = worker.generate_rollouts(True)
        attempt = 0
        while not success.any():
            rollout, success = worker.generate_rollouts(True)
            attempt += 1
#             print(worker.envs[0].env.goal)
            if attempt % 10 == 0:
                print("Trying iteration {} for worker {} on rollout {}".format(attempt, i, _))
            if attempt > max_attempt:
                rollout = None
                break
        worker_demos += [rollout]
    demo_rollouts += [worker_demos]
    


0
Trying iteration 10 for worker 0 on rollout 0
Trying iteration 20 for worker 0 on rollout 0
Trying iteration 30 for worker 0 on rollout 0
Trying iteration 40 for worker 0 on rollout 0
Trying iteration 50 for worker 0 on rollout 0
Trying iteration 60 for worker 0 on rollout 0
Trying iteration 70 for worker 0 on rollout 0
Trying iteration 80 for worker 0 on rollout 0
Trying iteration 90 for worker 0 on rollout 0
Trying iteration 100 for worker 0 on rollout 0
Trying iteration 10 for worker 0 on rollout 2
Trying iteration 20 for worker 0 on rollout 2
Trying iteration 30 for worker 0 on rollout 2
Trying iteration 40 for worker 0 on rollout 2
Trying iteration 50 for worker 0 on rollout 2
Trying iteration 60 for worker 0 on rollout 2
Trying iteration 70 for worker 0 on rollout 2
Trying iteration 80 for worker 0 on rollout 2
Trying iteration 90 for worker 0 on rollout 2
Trying iteration 100 for worker 0 on rollout 2
Trying iteration 10 for worker 0 on rollout 3
Trying iteration 20 for worker

Trying iteration 10 for worker 3 on rollout 8
Trying iteration 20 for worker 3 on rollout 8
4
Trying iteration 10 for worker 4 on rollout 0
Trying iteration 10 for worker 4 on rollout 3
Trying iteration 10 for worker 4 on rollout 4
Trying iteration 20 for worker 4 on rollout 4
Trying iteration 30 for worker 4 on rollout 4
Trying iteration 40 for worker 4 on rollout 4
Trying iteration 50 for worker 4 on rollout 4
Trying iteration 60 for worker 4 on rollout 4
Trying iteration 10 for worker 4 on rollout 5
Trying iteration 10 for worker 4 on rollout 6
Trying iteration 20 for worker 4 on rollout 6
Trying iteration 10 for worker 4 on rollout 8
Trying iteration 20 for worker 4 on rollout 8
Trying iteration 30 for worker 4 on rollout 8
Trying iteration 40 for worker 4 on rollout 8
Trying iteration 50 for worker 4 on rollout 8
Trying iteration 60 for worker 4 on rollout 8
Trying iteration 70 for worker 4 on rollout 8
Trying iteration 80 for worker 4 on rollout 8
Trying iteration 90 for worker 4

Trying iteration 40 for worker 8 on rollout 1
Trying iteration 10 for worker 8 on rollout 3
Trying iteration 10 for worker 8 on rollout 4
Trying iteration 10 for worker 8 on rollout 6
Trying iteration 10 for worker 8 on rollout 7
Trying iteration 20 for worker 8 on rollout 7
Trying iteration 30 for worker 8 on rollout 7
Trying iteration 40 for worker 8 on rollout 7
Trying iteration 50 for worker 8 on rollout 7
Trying iteration 10 for worker 8 on rollout 8
9
Trying iteration 10 for worker 9 on rollout 0
Trying iteration 20 for worker 9 on rollout 0
Trying iteration 30 for worker 9 on rollout 0
Trying iteration 40 for worker 9 on rollout 0
Trying iteration 50 for worker 9 on rollout 0
Trying iteration 60 for worker 9 on rollout 0
Trying iteration 10 for worker 9 on rollout 1
Trying iteration 20 for worker 9 on rollout 1
Trying iteration 30 for worker 9 on rollout 1
Trying iteration 40 for worker 9 on rollout 1
Trying iteration 10 for worker 9 on rollout 2
Trying iteration 20 for worker 9

In [98]:
# manually select rollout for now
n_obs_dim = demo_rollouts[0][1]['o'][0].shape[1]
n_act_dim = demo_rollouts[0][1]['u'][0].shape[1]

In [28]:
demo_rollouts[0][1]['ag'][0].shape

(51, 3)

In [29]:
demo_rollouts_path = os.path.join(data_dir, 'pilot_policy_demo_rollouts.pkl')

In [30]:
with open(os.path.join(data_dir, 'pilot_policy_demo_rollouts.pkl'), 'wb') as f:
  pickle.dump(demo_rollouts, f, pickle.HIGHEST_PROTOCOL)

In [31]:
# with open(demo_rollouts_path, 'rb') as f:
#   demo_rollouts = pickle.load(f)

# demo_rollouts = demo_rollouts[:len(train_goals)]

In [32]:
def build_mlp(
    input_placeholder,
    output_size,
    scope,
    n_layers=1,
    size=256,
    activation=tf.nn.relu,
    output_activation=None,
    reuse=False
  ):
  out = input_placeholder
  with tf.variable_scope(scope, reuse=reuse):
    for _ in range(n_layers):
      out = tf.layers.dense(out, size, activation=activation)
    out = tf.layers.dense(out, output_size, activation=output_activation)
  return out

In [89]:
def vectorize_rollouts(rollouts):
  obs = [[] for _ in range(n_train_tasks)]
  actions = [[] for _ in range(n_train_tasks)]
  next_obs = [[] for _ in range(n_train_tasks)]
  ach_goals = [[] for _ in range(n_train_tasks)]
  goals = [[] for _ in range(n_train_tasks)]
  for task_idx, task_rollouts in enumerate(rollouts):
    for task_rollout in task_rollouts:
        if task_rollout is None:
            continue
        more_obs, more_actions, more_ags = task_rollout['o'][0], task_rollout['u'][0], task_rollout['ag'][0]
        more_goals = task_rollout['g'][0]
        more_next_obs = more_obs[1:].copy()
        more_obs = more_obs[:-1]
        obs[task_idx] += [more_obs]
        actions[task_idx] += [more_actions]
        next_obs[task_idx] += [more_next_obs]
        ach_goals[task_idx] += [more_ags[1:]]
        goals[task_idx] += [more_goals]

  min_ro = len(min(obs, key=lambda o: len(o)))
  obs = np.array([np.vstack(obs_set[:min_ro]) for obs_set in obs])
  actions = np.array([np.vstack(action_set[:min_ro]) for action_set in actions])
  next_obs = np.array([np.vstack(obs_set[:min_ro]) for obs_set in next_obs])
  ach_goals = np.array([np.vstack(goal_set[:min_ro]) for goal_set in ach_goals])
  goals = np.array([np.vstack(goal_set[:min_ro]) for goal_set in goals])

  return obs, actions, next_obs, ach_goals, goals

In [90]:
demo_obs = None
demo_actions = None
demo_act_feats = None
demo_next_obs = None
demo_task_idxes = None
train_demo_example_idxes = None
val_demo_batch = None

In [91]:
def process_demo_rollouts(demo_rollouts):
  global demo_obs
  global demo_actions
  global demo_act_feats
  global demo_next_obs
  global demo_task_idxes
  global train_demo_example_idxes
  global val_demo_batch

  
  demo_obs, demo_actions, demo_next_obs, demo_ach_goals, demo_goals = vectorize_rollouts(demo_rollouts)
  demo_example_idxes = list(range(demo_obs.shape[1]))
  
  random.shuffle(demo_example_idxes)
  n_train_demo_examples = int(0.9 * len(demo_example_idxes))
  train_demo_example_idxes = demo_example_idxes[:n_train_demo_examples]
  val_demo_example_idxes = demo_example_idxes[n_train_demo_examples:]
  val_demo_batch = (demo_obs[:, val_demo_example_idxes], demo_actions[:, val_demo_example_idxes],
        demo_next_obs[:, val_demo_example_idxes], demo_ach_goals[:, val_demo_example_idxes], 
        demo_goals[:, val_demo_example_idxes])

In [92]:
process_demo_rollouts(demo_rollouts)

[(250, 3), (250, 3), (250, 3), (250, 3), (250, 3), (250, 3), (250, 3), (250, 3), (250, 3), (250, 3)]


In [94]:
demo_obs.shape

(10, 250, 25)

In [41]:
def sample_batch(size):
  idxes = random.sample(train_demo_example_idxes, size)
  demo_batch = demo_obs[:, idxes], demo_actions[:, idxes], demo_act_feats[:, idxes], demo_next_obs[:, idxes]
  return demo_batch

In [ ]:
gamma = 0.99
iterations = 100000
learning_rate = 1e-3
batch_size = 512 // n_train_tasks
sq_td_err_penalty = 1e-3
trans_err_penalty = 1e0


q_n_layers = 1
q_layer_size = 32
q_activation = tf.nn.relu
q_output_activation = None

invdyn_n_layers = 1
invdyn_layer_size = 32
invdyn_activation = tf.nn.relu
invdyn_output_activation = None


constraint_sampling_freq = 100000
constraint_batch_size = batch_size
n_constraint_rollouts_per_env = 100

val_update_freq = 100

In [95]:
# im_scope =  str(uuid.uuid4())
# q_scope = str(uuid.uuid4())
# invdyn_scope = str(uuid.uuid4())

# with open(os.path.join(data_dir, 'im_scope.pkl'), 'wb') as f:
#   pickle.dump(im_scope, f)

# with open(os.path.join(data_dir, 'q_scope.pkl'), 'wb') as f:
#   pickle.dump(q_scope, f)

# with open(os.path.join(data_dir, 'invdyn_scope.pkl'), 'wb') as f:
#   pickle.dump(invdyn_scope, f)

In [ ]:
with open(os.path.join(data_dir, 'im_scope.pkl'), 'rb') as f:
  im_scope = pickle.load(f)

with open(os.path.join(data_dir, 'q_scope.pkl'), 'rb') as f:
  q_scope = pickle.load(f)

with open(os.path.join(data_dir, 'invdyn_scope.pkl'), 'rb') as f:
  invdyn_scope = pickle.load(f)

In [99]:
demo_obs_t_ph = tf.placeholder(tf.float32, [n_train_tasks, None, n_obs_dim], name="dot")
demo_act_t_ph = tf.placeholder(tf.int32, [n_train_tasks, None, n_act_dim], name="dat")
demo_next_obs_t_ph = tf.placeholder(tf.float32, [n_train_tasks, None, n_obs_dim], name="dnot")
demo_batch_size_ph = tf.placeholder(tf.int32, name="dbs")

sample_act_t_ph = tf.placeholder(tf.int32, [n_train_tasks, None, n_act_dim, n_act_samples], name="sat")


constraint_obs_t_ph = tf.placeholder(tf.float32, [n_train_tasks, None, n_obs_dim], name="cot")
constraint_act_t_ph = tf.placeholder(tf.float32, [n_train_tasks, None, n_act_dim], name="cat")
constraint_rew_t_ph = tf.placeholder(tf.float32, [n_train_tasks, None], name="crt")
constraint_batch_size_ph = tf.placeholder(tf.int32, name="cbs")

In [ ]:
demo_batch_idxes = tf.reshape(
  tf.range(0, demo_batch_size_ph, 1), 
  [demo_batch_size_ph, 1])

extract_task = lambda x, i: tf.squeeze(tf.gather(x, tf.convert_to_tensor(
  [i], dtype=tf.int32)), axis=[0]) 

demo_q_t = tf.stack([tf.gather_nd(
  build_mlp(
    tf.concat((extract_task(demo_obs_t_ph, train_task_idx),  extract_task(demo_act_t_ph, train_task_idx)), axis=1),
    n_act_dim, q_scope+'-'+str(train_task_idx), 
    n_layers=q_n_layers, size=q_layer_size,
    activation=q_activation, output_activation=q_output_activation
  ), 
  tf.concat([
    demo_batch_idxes, 
    tf.expand_dims(extract_task(demo_act_t_ph, train_task_idx), 1)], axis=1)
) for train_task_idx in range(n_train_tasks)], axis=0)


demo_v_t_inputs = []
for i in range(n_train_tasks):
    obs = tf.tile(tf.expand_dims(extract_task(demo_obs_t_ph, train_task_idx),-1), [1,1,n_act_samples])
    demo_v_t_inputs += [inp]
    

demo_v_t = tf.reduce_logsumexp(
  tf.stack([build_mlp(
    tf.concat((extract_task(demo_obs_t_ph, train_task_idx),  extract_task(sample_act_t_ph, train_task_idx)), axis=1),
    n_act_dim, q_scope+'-'+str(train_task_idx), 
    n_layers=q_n_layers, size=q_layer_size,
    activation=q_activation, output_activation=q_output_activation,
    reuse=True
  ) for train_task_idx in range(n_train_tasks)], axis=0),
  axis=2)

act_log_likelihoods = demo_q_t - demo_v_t

In [ ]:
neg_avg_log_likelihood = -tf.reduce_mean(act_log_likelihoods)

In [ ]:
pred_next_obs = build_mlp(
  tf.concat((demo_obs_t_ph, demo_act_t_ph), axis=2),
  n_obs_dim, invdyn_scope, 
  n_layers=invdyn_n_layers, size=invdyn_layer_size,
  activation=invdyn_activation, output_activation=invdyn_output_activation)

In [ ]:
demo_next_obs_t_reshaped = tf.reshape(
  demo_next_obs_t_ph, [n_train_tasks, demo_batch_size_ph, n_obs_feats])

demo_pred_next_obs_t_reshaped = tf.reshape(
    pred_next_obs, [n_train_tasks, demo_batch_size_ph, n_obs_feats])

In [ ]:
trans_err = pred_next_obs - demo_next_obs_t_ph

In [ ]:
trans_loss = tf.reduce_mean(trans_err**2)

In [ ]:
# Predicted constraint next state given inv dyns
constraint_obs_tp1 = tf.reshape(build_mlp(
  tf.concat((constraint_obs_t_ph, constraint_act_t_feats_ph), axis=2),
  n_obs_dim, invdyn_scope, 
  n_layers=invdyn_n_layers, size=invdyn_layer_size,
  activation=invdyn_activation, output_activation=invdyn_output_activation,
  reuse=True), [n_train_tasks, constraint_batch_size_ph, n_obs_feats])

In [ ]:
q_tp1 = tf.stack([build_mlp(
    extract_task(constraint_obs_tp1, train_task_idx), 
    n_act_dim, q_scope+'-'+str(train_task_idx),
    n_layers=q_n_layers, size=q_layer_size,
    activation=q_activation, output_activation=q_output_activation, 
    reuse=True) for train_task_idx in range(n_train_tasks)], axis=0)
v_tp1 = tf.reduce_logsumexp(q_tp1, axis=2)

In [ ]:
target_t = constraint_rew_t_ph + gamma * v_tp1

In [ ]:
constraint_batch_idxes = tf.reshape(
  tf.range(0, constraint_batch_size_ph, 1), 
  [constraint_batch_size_ph, 1])

# Sampled constraint state q-vals
q_t = tf.stack([tf.gather_nd(
  build_mlp(
    extract_task(constraint_obs_t_ph, train_task_idx), 
    n_act_dim, q_scope+'-'+str(train_task_idx), 
    n_layers=q_n_layers, size=q_layer_size,
    activation=q_activation, output_activation=q_output_activation, 
    reuse=True
  ), 
  tf.concat([
    constraint_batch_idxes, 
    tf.expand_dims(extract_task(constraint_act_t_ph, train_task_idx), 1)], axis=1)
) for train_task_idx in range(n_train_tasks)], axis=0)

In [ ]:
# Bellman errors
td_err = q_t - target_t

In [ ]:
sq_td_err = tf.reduce_mean(td_err**2)

In [ ]:
loss = neg_avg_log_likelihood + sq_td_err_penalty * sq_td_err + trans_err_penalty * trans_loss

In [ ]:
full_update_op = tf.train.AdamOptimizer(learning_rate).minimize(loss)
trans_update_op = tf.train.AdamOptimizer(learning_rate).minimize(trans_loss)

In [ ]:
def sample_constraints(_):
  constraint_rollouts = [[] for _ in range(n_train_tasks)]
  
  for train_task_idx in range(n_train_tasks):
    rollouts = [[] for _ in range(n_constraint_rollouts_per_env)]
    envs = [make_lander_env(
      fps=fps, goal=train_goals[train_task_idx]) for _ in range(
      n_constraint_rollouts_per_env)]
    obses = np.array([env.reset() for env in envs])
    dones = [False for _ in envs]
    prev_obses = obses
    for step_idx in range(max_ep_len+1):
      not_done_idxes = [i for i, done in enumerate(dones) if not done]
      batch_size = len(not_done_idxes)
      if batch_size == 0:
        break
      actions = np.random.choice(n_act_dim, batch_size)
      for i, env_idx in enumerate(not_done_idxes):
        env = envs[env_idx]
        action = actions[i]
        obs, r, done, info = env.step(action)
        obses[env_idx] = obs
        dones[env_idx] = done
        rollouts[env_idx].append((
          prev_obses[env_idx], action, r))
      prev_obses = copy(obses)
    constraint_rollouts[train_task_idx].extend([r for r in rollouts if r != []])

  size = min(sum(len(r) for r in rollouts) for rollouts in constraint_rollouts)
  
  global train_constraint_example_idxes
  global val_constraint_batch
  global constraint_obs_t
  global constraint_act_t
  global constraint_rew_t
  global constraint_act_t_feats
    
  constraint_obs_t = np.zeros((n_train_tasks, size, n_obs_feats))
  constraint_act_t = np.zeros((n_train_tasks, size))
  constraint_act_t_feats = np.zeros((n_train_tasks, size, n_act_feats))
  constraint_rew_t = np.zeros((n_train_tasks, size))
  
  for train_task_idx in range(n_train_tasks):
    unfeat_obses, actions, rews = list(zip(*sum(
      constraint_rollouts[train_task_idx], [])))
    obses = [featurize_obs(s) for s in unfeat_obses]
    act_feats = [disc_to_cont(a) for a in actions]
    idxes = random.sample(list(range(len(obses))), size)
    constraint_obs_t[train_task_idx, :, :] = np.array(obses)[idxes, :]
    constraint_act_t[train_task_idx, :] = np.array(actions)[idxes]
    constraint_rew_t[train_task_idx, :] = np.array(rews)[idxes]
    constraint_act_t_feats[train_task_idx, :, :] = np.array(act_feats)[idxes, :]

  
  constraint_example_idxes = list(range(size))
  random.shuffle(constraint_example_idxes)
  n_train_constraint_examples = int(0.9 * size)
  
  train_constraint_example_idxes = constraint_example_idxes[:n_train_constraint_examples]
  val_constraint_example_idxes = constraint_example_idxes[n_train_constraint_examples:]
  val_constraint_batch = (constraint_obs_t[:, val_constraint_example_idxes],
                          constraint_act_t[:, val_constraint_example_idxes],
                          constraint_act_t_feats[:, val_constraint_example_idxes],
                          constraint_rew_t[:, val_constraint_example_idxes])

In [ ]:
def sample_constraint_batch(size):
  global n_iters_since_prev_constraint_sample
  if n_iters_since_prev_constraint_sample % constraint_sampling_freq == 0:
    sample_constraints(size)
    n_iters_since_prev_constraint_sample = 0
  n_iters_since_prev_constraint_sample += 1

  idxes = random.sample(train_constraint_example_idxes, size)
  constraint_batch = (constraint_obs_t[:, idxes],
    constraint_act_t[:, idxes],
    constraint_act_t_feats[:, idxes],
    constraint_rew_t[:, idxes])
  return constraint_batch

In [ ]:
train_constraint_example_idxes = None
val_constraint_batch = None
constraint_obs_t = None
constraint_act_t = None
constraint_act_t_feats = None
constraint_rew_t = None
n_iters_since_prev_constraint_sample = 0

In [ ]:
with open(os.path.join(data_dir, 'constraint_samples.pkl'), 'wb') as f:
  pickle.dump((
    train_constraint_example_idxes, 
    val_constraint_batch,
    constraint_obs_t,
    constraint_act_t,
    constraint_act_t_feats,
    constraint_rew_t,
    n_iters_since_prev_constraint_sample), f, pickle.HIGHEST_PROTOCOL)

In [ ]:
with open(os.path.join(data_dir, 'constraint_samples.pkl'), 'rb') as f:
  (train_constraint_example_idxes, 
    val_constraint_batch,
    constraint_obs_t,
    constraint_act_t,
    constraint_act_t_feats,
    constraint_rew_t,
    n_iters_since_prev_constraint_sample) = pickle.load(f)

In [ ]:
tf.global_variables_initializer().run(session=sess)

In [ ]:
n_iters = iterations * demo_obs.shape[1] // batch_size

In [ ]:
def compute_batch_loss(demo_batch, constraint_batch, step=False, trans_only=False, t=None):
  demo_batch_obs_t, demo_batch_act_t, demo_batch_act_t_feats, demo_batch_next_obs_t = demo_batch
  constraint_batch_obs_t, constraint_batch_act_t, constraint_batch_act_t_feats, constraint_batch_rew_t = constraint_batch

  feed_dict = {
    demo_obs_t_ph: demo_batch_obs_t,
    demo_act_t_ph: demo_batch_act_t,
    demo_act_t_feats_ph: demo_batch_act_t_feats,
    demo_next_obs_t_ph: demo_batch_next_obs_t,
    demo_batch_size_ph: demo_batch_obs_t.shape[1],
    constraint_obs_t_ph: constraint_batch_obs_t,
    constraint_act_t_ph: constraint_batch_act_t,
    constraint_act_t_feats_ph: constraint_batch_act_t_feats,
    constraint_rew_t_ph: constraint_batch_rew_t,
    constraint_batch_size_ph: constraint_batch_obs_t.shape[1],
  }
  
  if trans_only:
    [loss_eval] = sess.run([trans_loss], feed_dict=feed_dict)
    update_op = trans_update_op
    d = {
    'loss': loss_eval,
      }
  else:
    [loss_eval, neg_avg_log_likelihood_eval, sq_td_err_eval, trans_loss_eval] = sess.run(
        [loss, neg_avg_log_likelihood, sq_td_err, trans_loss], feed_dict=feed_dict)
    update_op = full_update_op
    d = {
        'loss': loss_eval,
        'nll': neg_avg_log_likelihood_eval,
        'ste': sq_td_err_eval,
        'tl': trans_loss_eval
      }
  
  if step:
    sess.run(update_op, feed_dict=feed_dict)
#   else:
#     d.update(compute_int_dyn_nll())
    
#   print(sess.run([q_t], feed_dict=feed_dict)[0].shape)
#   print(sess.run([target_t], feed_dict=feed_dict)[0].shape)


  return d

## Learning Dynamics with Reward

In [ ]:
full_train_logs = {
  'loss_evals': [],
  'tl_evals': [],
  'nll_evals': [],
  'ste_evals': [],
  'val_loss_evals': [],
  'val_tl_evals': [],
  'val_nll_evals': [],
  'val_ste_evals': [],
}

In [ ]:
val_log = None
while len(full_train_logs['loss_evals']) < n_iters:
  demo_batch = sample_batch(batch_size)
  constraint_batch = sample_constraint_batch(constraint_batch_size)
  t = len(full_train_logs['loss_evals'])
  train_log = compute_batch_loss(demo_batch, constraint_batch, step=True, t=t)
  if val_log is None or len(full_train_logs['loss_evals']) % val_update_freq == 0:
    val_log = compute_batch_loss(val_demo_batch, val_constraint_batch, step=False, t=t)
  
  if len(full_train_logs['loss_evals']) % 1000 == 0:
      print('%d %d %f %f %f %f' % (
        t, n_iters, train_log['loss'], train_log['tl'], val_log['loss'], val_log['tl'])
      )
  for k, v in train_log.items():
    full_train_logs['%s_evals' % k].append(v)
  for k, v in val_log.items():
    full_train_logs['%s%s_evals' % ('val_' if k in ['loss', 'nll', 'ste', 'tl'] else '', k)].append(v)
  if len(full_train_logs['loss_evals']) % 10000 == 0:
    for k in ['val_nll_evals', 'val_ste_evals']:
      plt.xlabel('Iterations')
      plt.ylabel(k.split('_')[1])
      plt.plot(full_train_logs[k])
      plt.show()

In [ ]:
for k in ['val_nll_evals', 'val_ste_evals']:
  plt.xlabel('Iterations')
  plt.ylabel(k.split('_')[1])
  plt.plot(full_train_logs[k])
  plt.show()

In [ ]:
invdyn_r_path = os.path.join(data_dir, 'invdyn_r.tf')

In [ ]:
# save_tf_vars(sess, invdyn_scope, invdyn_r_path)
load_tf_vars(sess, invdyn_scope, invdyn_r_path)

In [ ]:
with open(os.path.join(data_dir, 'qt_results.pkl'), 'wb') as f:
  pickle.dump(full_train_logs, f)

## Learning Dynamics Without Reward

In [ ]:
train_constraint_example_idxes = None
val_constraint_batch = None
constraint_obs_t = None
constraint_act_t = None
constraint_act_t_feats = None
n_iters_since_prev_constraint_sample = 0

tf.global_variables_initializer().run(session=sess)

n_iters = iterations * demo_obs.shape[1] // batch_size

trans_train_logs = {
  'loss_evals': [],
  'nll_evals': [],
  'ste_evals': [],
  'val_loss_evals': [],
  'val_nll_evals': [],
  'val_ste_evals': [],
  'int_dyn_err_evals': []
}
val_log = None

In [ ]:
while len(trans_train_logs['loss_evals']) < n_iters:
  demo_batch = sample_batch(batch_size)
  constraint_batch = sample_constraint_batch(constraint_batch_size)
  
  t = len(trans_train_logs['loss_evals'])
  train_log = compute_batch_loss(demo_batch, constraint_batch, trans_only=True, step=True, t=t)
  if val_log is None or len(trans_train_logs['loss_evals']) % val_update_freq == 0:
    val_log = compute_batch_loss(val_demo_batch, val_constraint_batch, trans_only=True, step=False, t=t)
  
  if len(trans_train_logs['loss_evals']) % 1000 == 0:
      print('%d %d %f %f' % (
        t, n_iters, train_log['loss'], val_log['loss'],)
      )

  trans_train_logs['loss_evals'].append(train_log['loss'])
  trans_train_logs['val_loss_evals'].append(val_log['loss'])
  if len(trans_train_logs['loss_evals']) % 10000 == 0:
    for k in ['loss_evals', 'val_loss_evals']:
      plt.xlabel('Iterations')
      plt.ylabel(k.split('_')[1])
      plt.plot(trans_train_logs[k])
      plt.show()

In [ ]:
invdyn_nr_path = os.path.join(data_dir, 'invdyn_nr.tf')

In [ ]:
save_tf_vars(sess, invdyn_scope, invdyn_nr_path)

# Use Q-vals for policy

In [ ]:
train_task_idx = 3
train_env = train_envs[train_task_idx]
obs_t_ph = tf.placeholder(tf.float32, [None, n_obs_feats], name="ot")

training_qs = build_mlp(
    obs_t_ph,
    n_act_dim, q_scope+'-'+str(train_task_idx), 
    n_layers=q_n_layers, size=q_layer_size,
    activation=q_activation, output_activation=q_output_activation,
    reuse=True)

policy_action = tf.argmax(training_qs, axis=1)

In [ ]:
 def run_q_pol_ep(policy_action, obs_ph, env, max_ep_len=max_ep_len, render=False, task_idx=None):
  obs = env.reset()
  done = False
  totalr = 0.
  prev_obs = obs
  rollout = []
  for step_idx in range(max_ep_len+1):
    if done:
      break
    action = sess.run(policy_action, feed_dict={obs_ph:np.array([obs])})[0]
    obs, r, done, info = env.step(action)
    rollout.append((prev_obs, action, r, obs, float(done), task_idx))
    prev_obs = obs
    if render:
      env.render()
    totalr += r
  return rollout

In [ ]:
%autoreload
run_q_pol_ep(policy_action, obs_t_ph, train_env, render=True, task_idx=train_task_idx)

In [ ]:
train_envs[train_task_idx].close()

# Train Q-vals for arbitrary goals using Bellman Residual

In [ ]:
obs_t_ph = tf.placeholder(tf.float32, [None, n_obs_feats])
act_t_ph = tf.placeholder(tf.int32, [None])
act_t_feats_ph = tf.placeholder(tf.float32, [None, n_act_feats])
next_obs_t_ph = tf.placeholder(tf.float32, [None, n_obs_feats])
batch_size_ph = tf.placeholder(tf.int32)


con_obs_t_ph = tf.placeholder(tf.float32, [None, n_obs_feats])
con_act_t_ph = tf.placeholder(tf.int32, [None])
con_act_t_feats_ph = tf.placeholder(tf.float32, [None, n_act_feats])
con_rew_t_ph = tf.placeholder(tf.float32, [None])
con_batch_size_ph = tf.placeholder(tf.int32)

In [ ]:
def softmax(x, axis=1):
    """Compute softmax values for each sets of scores in x."""
    axis = 0 if len(x.shape) == 1 else axis
    e_x = np.exp(x - np.max(x, axis=axis,keepdims=True))
    return (e_x / e_x.sum(axis=axis, keepdims = True))

def sample_q_learn_constraints(goal, n_samples=n_constraint_rollouts_per_env, q_fn = None):
  constraint_rollouts = []
  rollouts = [[] for _ in range(n_samples)]
  envs = [make_lander_env(goal=goal) for _ in range(n_samples)]
  obses = np.array([env.reset() for env in envs])
  dones = [False for _ in envs]
  prev_obses = obses
  for step_idx in range(max_ep_len+1):
    not_done_idxes = [i for i, done in enumerate(dones) if not done]
    batch_size = len(not_done_idxes)
    if batch_size == 0:
      break
    if q_fn is None:
        actions = np.random.choice(n_act_dim, batch_size)
    else:
        action_probs = sess.run([q_fn], feed_dict={obs_t_ph: np.array(prev_obses)})[0]
        actions = np.array([np.random.choice(n_act_dim, p=softmax(aps)) for aps in action_probs])
    for i, env_idx in enumerate(not_done_idxes):
      env = envs[env_idx]
      action = actions[i]
      obs, r, done, info = env.step(action)
      obses[env_idx] = obs
      dones[env_idx] = done
      rollouts[env_idx].append((prev_obses[env_idx], action, r))
      prev_obses = copy(obses)
    constraint_rollouts.extend([r for r in rollouts if r != []])

  size = sum(len(r) for r in rollouts)
    
  unfeat_obses, actions, rews = list(zip(*sum(constraint_rollouts, [])))
  obses = [featurize_obs(s) for s in unfeat_obses]
  act_feats = [disc_to_cont(a) for a in actions]
  idxes = random.sample(list(range(len(obses))), size)
  constraint_obs_t = np.array(obses)[idxes, :]
  constraint_act_t = np.array(actions)[idxes]
  constraint_rew_t = np.array(rews)[idxes]
  constraint_act_t_feats = np.array(act_feats)[idxes, :]

  
  constraint_example_idxes = list(range(size))
  random.shuffle(constraint_example_idxes)
  n_train_constraint_examples = int(0.9 * size)
  
  train_constraint_example_idxes = constraint_example_idxes[:n_train_constraint_examples]
  train_constraint_batch = [constraint_obs_t[train_constraint_example_idxes],
                            constraint_act_t[train_constraint_example_idxes],
                            constraint_act_t_feats[train_constraint_example_idxes],
                            constraint_rew_t[train_constraint_example_idxes]]
    
  val_constraint_example_idxes = constraint_example_idxes[n_train_constraint_examples:]
  val_constraint_batch = (constraint_obs_t[val_constraint_example_idxes],
                          constraint_act_t[val_constraint_example_idxes],
                          constraint_act_t_feats[val_constraint_example_idxes],
                          constraint_rew_t[val_constraint_example_idxes])

  return train_constraint_batch, val_constraint_batch

In [ ]:
def sample_q_learn_constraint_batch(size):
  global train_constraints
  train_constraint_example_idxes = list(range(len(train_constraints[0])))
  idxes = random.sample(train_constraint_example_idxes, size)
  constraint_batch = tuple(map(lambda x: x[idxes], train_constraints))
  return constraint_batch

In [ ]:
train_id = 9
qlearn_env = make_lander_env(goal=train_id)

In [ ]:
train_constraints[3].shape

In [ ]:
con_obs_tp1 = build_mlp(
  tf.concat((con_obs_t_ph, con_act_t_feats_ph), axis=1),
  n_obs_dim, invdyn_scope, 
  n_layers=invdyn_n_layers, size=invdyn_layer_size,
  activation=invdyn_activation, output_activation=invdyn_output_activation,
  reuse=True)

In [ ]:
# Sampled constraint state q-vals
train_q_t = build_mlp(
    con_obs_t_ph,
    n_act_dim, q_scope+'-'+str(train_id), 
    n_layers=q_n_layers, size=q_layer_size,
    activation=q_activation, output_activation=q_output_activation)

In [ ]:
pol_q_t = build_mlp(
    obs_t_ph,
    n_act_dim, q_scope+'-'+str(train_id), 
    n_layers=q_n_layers, size=q_layer_size,
    activation=q_activation, output_activation=q_output_activation,
    reuse=True)

In [ ]:
train_q_tp1 = build_mlp(
    con_obs_tp1, 
    n_act_dim, q_scope+'-'+str(train_id),
    n_layers=q_n_layers, size=q_layer_size,
    activation=q_activation, output_activation=q_output_activation,
    reuse=True)

train_v_tp1 = tf.reduce_logsumexp(train_q_tp1, axis=1)

In [ ]:
train_target_t = con_rew_t_ph + gamma * train_v_tp1

In [ ]:
# Bellman errors
train_td_err = tf.transpose(tf.transpose(train_q_t) - train_target_t)
train_sq_td_err = tf.reduce_mean(train_td_err**2)

In [ ]:
q_train_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, q_scope+'-'+str(train_id))

In [ ]:
train_update_op = tf.train.AdamOptimizer(learning_rate).minimize(train_sq_td_err, var_list=q_train_vars)

In [ ]:
def compute_qbatch_loss(constraint_batch, update_op, step=True):
  constraint_batch_obs_t, constraint_batch_act_t, constraint_batch_act_t_feats, constraint_batch_rew_t = constraint_batch

  feed_dict = {
    con_obs_t_ph: constraint_batch_obs_t,
    con_act_t_ph: constraint_batch_act_t,
    con_act_t_feats_ph: constraint_batch_act_t_feats,
    con_rew_t_ph: constraint_batch_rew_t,
    con_batch_size_ph: constraint_batch_obs_t.shape[1],
  }

  sq_td_err_eval = sess.run([train_sq_td_err], feed_dict=feed_dict)[0]
  
  if step:
      sess.run(update_op, feed_dict=feed_dict)
    
  return sq_td_err_eval

In [ ]:
train_constraints, val_constraints = sample_q_learn_constraints(train_id)

In [ ]:
tf.global_variables_initializer().run(session=sess)
load_tf_vars(sess, invdyn_scope, invdyn_r_path)

In [ ]:
soft_train_logs = {
  'err': [],
  'val_err':[]
}
max_iters = 1e7

In [ ]:
val_err = None
while len(soft_train_logs['err']) < max_iters:
  constraint_batch = sample_q_learn_constraint_batch(constraint_batch_size)
  t = len(soft_train_logs['err'])
  err = compute_qbatch_loss(constraint_batch, train_update_op)
  if val_err is None or len(soft_train_logs['val_err']) % val_update_freq == 0:
    val_err = compute_qbatch_loss(val_constraints, train_update_op, step=False)
  if len(soft_train_logs['err']) % 1000 == 0:
      print('%d %d %f %f' % (t, n_iters, err, val_err))
  soft_train_logs['err'].append(err)
  soft_train_logs['val_err'].append(val_err)
  if len(soft_train_logs['err']) % 10000 == 0:
    plt.xlabel('Iterations')
    plt.ylabel("Validation Residual Error")
    plt.plot(soft_train_logs['val_err'])
    plt.show()

In [ ]:
soft_train_logs = {
  'err': [],
  'val_errs':[[]]
}
max_iters = 1e6

val_errs = []
dagger_iter_rate = 2e4
dagger_constraint_sims = 10
val_constraint_list = [val_constraints]

In [ ]:
while len(soft_train_logs['err']) < max_iters:
  if len(soft_train_logs['err']) % dagger_iter_rate == 0 and len(soft_train_logs['err']) != 0:
    print("Generating new rollouts")
    new_train_cons, new_val_cons = sample_q_learn_constraints(train_id, dagger_constraint_sims, pol_q_t)
    n_new_cons = new_train_cons[0].shape[0]
    for i in range(len(train_constraints)):
        print(train_constraints[i].shape, new_train_cons[i].shape)
        if len(train_constraints[i].shape) == 2:
            train_constraints[i] = np.vstack((train_constraints[i], new_train_cons[i]))[n_new_cons:]
        else:
            train_constraints[i] = np.append(train_constraints[i], new_train_cons[i])[n_new_cons:]
    val_constraint_list += [new_val_cons]
    soft_train_logs['val_errs'] += [[]]
  constraint_batch = sample_q_learn_constraint_batch(constraint_batch_size)
  t = len(soft_train_logs['err'])
  err = compute_qbatch_loss(constraint_batch, train_update_op)
  if len(val_errs) == 0 or len(soft_train_logs['val_errs'][0]) % val_update_freq == 0:
    for i,val_cons in enumerate(val_constraint_list):
        val_err = compute_qbatch_loss(val_cons, train_update_op, step=False)
        soft_train_logs['val_errs'][i].append(val_err)

  if len(soft_train_logs['err']) % 1000 == 0:
      print('%d %d %f %f' % (t, n_iters, err, val_err))
  soft_train_logs['err'].append(err)
  if len(soft_train_logs['err']) % 10000 == 0:
    plt.xlabel('Iterations')
    plt.ylabel("Validation Residual Error")
    for val_err in soft_train_logs['val_errs']:
        plt.plot(val_err)
    plt.show()

In [ ]:
def q_policy(obs):
    policy_action = tf.argmax(pol_q_t, axis=1)
    return sess.run([policy_action], feed_dict = {obs_t_ph : np.array([obs])})[0][0]


In [ ]:
run_ep(q_policy, train_env, max_ep_len=max_ep_len, render=True, task_idx=None)
qlearn_env.close()

# MPC-R attempt [failed due to difficulties with contactlisteners]

In [ ]:
obs_t_ph = tf.placeholder(tf.float32, [None, n_obs_feats], name="ot")
act_t_feats_ph = tf.placeholder(tf.float32, [None, n_act_feats], name="atf")

obs_tp1 = build_mlp(
  tf.concat((obs_t_ph, act_t_feats_ph), axis=1),
  n_obs_dim, invdyn_scope, 
  n_layers=invdyn_n_layers, size=invdyn_layer_size,
  activation=invdyn_activation, output_activation=invdyn_output_activation,
  reuse=True)

In [ ]:
controller = MPCController(train_env, train_env.env._reward, obs_t_ph, act_t_feats_ph, obs_tp1, sess, 100, 50, gamma, n_act_dim, featurize_act)

In [ ]:
def run_controller_ep(policy, env, max_ep_len=max_ep_len, render=False, task_idx=None):
  obs = env.reset()
  done = False
  totalr = 0.
  prev_obs = obs
  rollout = []
  for step_idx in range(max_ep_len+1):
    if done:
      break
    action = policy(obs, step_idx)
    obs, r, done, info = env.step(action)
    rollout.append((prev_obs, action, r, obs, float(done), task_idx))
    prev_obs = obs
    if render:
      env.render()
    totalr += r
  return rollout

In [ ]:
%autoreload
run_controller_ep(controller.get_action, train_env, max_ep_len=max_ep_len, render=False, task_idx=None)

In [ ]:
train_env.close()

In [ ]:
import cProfile

def do_cprofile(func):
    def profiled_func(*args, **kwargs):
        profile = cProfile.Profile()
        try:
            profile.enable()
            result = func(*args, **kwargs)
            profile.disable()
            return result
        finally:
            profile.print_stats()
    return profiled_func


profiled_run_ep = do_cprofile(run_controller_ep)

In [ ]:
profiled_run_ep(controller.get_action, train_env, max_ep_len=max_ep_len, render=True, task_idx=None)

In [ ]:
train_env.close()

In [ ]:
from line_profiler import LineProfiler

def do_profile(follow=[]):
    def inner(func):
        def profiled_func(*args, **kwargs):
            try:
                profiler = LineProfiler()
                profiler.add_function(func)
                for f in follow:
                    profiler.add_function(f)
                profiler.enable_by_count()
                return func(*args, **kwargs)
            finally:
                profiler.print_stats()
        return profiled_func
    return inner


In [ ]:
@do_profile(follow=[train_env.env._reward,controller.get_action])
def lp_run_controller_ep(policy, env, max_ep_len=max_ep_len, render=False, task_idx=None):
  obs = env.reset()
  done = False
  totalr = 0.
  prev_obs = obs
  rollout = []
  for step_idx in range(max_ep_len+1):
    if done:
      break
    action = policy(obs, step_idx)
    obs, r, done, info = env.step(action)
    rollout.append((prev_obs, action, r, obs, float(done), task_idx))
    prev_obs = obs
    if render:
      env.render()
    totalr += r
  return rollout

In [ ]:
lp_run_controller_ep(controller.get_action, train_env, max_ep_len=max_ep_len, render=False, task_idx=None)